In [ ]:
from PIL import Image
import numpy as np
import math
import cv2

imgPath="../assest/lean.jpg"
src_img=np.array(Image.open(imgPath).convert('L')) #numpy.ndarray (512,512)
def test():
 angle=45
 d=math.radians(angle)
 print(math.sin(d),math.cos(d))

 #print(matrix)
def shift(shiftValue):
 matrix=np.zeros((3,3),dtype=np.uint8)
 for i in range(0,3):
  matrix[:,2]=shiftValue
  matrix[i,i]=1
 dst_img=np.zeros((1024,1024))
 for x in range(0,10):
  for y in range(0,10):
   src_index=np.array([x,y,1],ndmin=2).T
   dst_index=np.matmul(matrix,src_index)
   print(dst_index)
   if dst_index[0,0] in range(0,1024) and dst_index[1,0] in range(0,1024):
     dst_img[int(dst_index[0,0]),int(dst_index[1,0])]=src_img[x,y]
 result=Image.fromarray(dst_img.astype('uint8'))
 result.save("shiftResult.jpg")
 print("图像移动:",shiftValue)

# 目前存在的问题，当你坐标转化之后为小数，而你定义的矩阵精度是int，出现了bug
def rotate(angle):
 matrix=np.zeros((3,3),dtype=np.ulonglong)
 matrix[0,0]=0
 matrix[0,1]=-1
 matrix[1,0]=1
 matrix[1,1]=0
 matrix[2,2]=1
 result=trans(matrix)
 #print(result[:, :])
 # result.save("test.jpg")
def mirror():
 pass
def trans(matrix):
 dst_img=np.zeros((1024,1024))
 for x in range(0,10):
  for y in range(0,10):
   src_index=np.array([x,y,1],ndmin=2).T
   dst_index=np.matmul(matrix,src_index)
   print(dst_index)
   if dst_index[0,0] in range(0,1024) and dst_index[1,0] in range(0,1024):
    dst_img[int(dst_index[0,0]),int(dst_index[1,0])]=src_img[x,y]
 # result=Image.fromarray(dst_img.astype('uint8'))
 result = dst_img
 return result
rotate(100)
# shift(100)